In [19]:
ids, channels, data, sampling_frequencies, reference_systems, eeg_labels = load_references('../../shared_data/training_mini/') 
# Importiere EKG-Dateien, zugehörige Diagnose, Sampling-Frequenz (Hz) und Name (meist fs=256 Hz)

100	 Dateien wurden geladen.


In [20]:
# -*- coding: utf-8 -*-

import csv
import matplotlib.pyplot as plt
import numpy as np
import os
from wettbewerb import load_references, get_3montages
import mne
from scipy import signal as sig
import ruptures as rpt
import json
import pywt
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import joblib
from sklearn.metrics import roc_curve, auc
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from imblearn.over_sampling import SMOTE

import numpy as np
from wettbewerb import load_references, get_3montages, get_6montages
import pywt
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE

number_montages = 3
N_samples = 2000 # Number of samples per division
# Decompose the wave
wavelet = 'db4'
scaler = StandardScaler()
new_signal = []

mont1_signal = []
mont2_signal = []
mont3_signal = []
whole_mont = [mont1_signal,mont2_signal,mont3_signal]
for i,_id in enumerate(ids):
    
    if number_montages == 6:
        _montage, _montage_data, _is_missing = get_6montages(channels[i], data[i])
    else:
        _montage, _montage_data, _is_missing = get_3montages(channels[i], data[i])
        
    _fs = sampling_frequencies[i]
    features_per_id = []

    for j, signal_name in enumerate(_montage):
        signal = _montage_data[j]
        # Notch-Filter to compensate net frequency of 50 Hz
        signal_notch = mne.filter.notch_filter(x=signal, Fs=_fs, freqs=np.array([50.,100.]), n_jobs=2, verbose=False)
        # Bandpassfilter between 0.5Hz and 70Hz to filter out noise
        signal_filter = mne.filter.filter_data(data=signal_notch, sfreq=_fs, l_freq=0.5, h_freq=70.0, n_jobs=2, verbose=False)
        # Defining number of divisions for signal
        N_div = len(signal_filter)//N_samples
        # Normalizing data
        norm_montage_data = scaler.fit_transform(signal_filter.reshape(-1,1)).reshape(1,-1)[0]
    
        for i in range(N_div):
            montage_array = norm_montage_data[i*N_samples:(i+1)*N_samples]
            whole_mont[j].append(montage_array)


labels = []
for i,_id in enumerate(ids):
    if eeg_labels[i][0]:
        onset = eeg_labels[i][1]
        offset = eeg_labels[i][2]
        sample_freq = sampling_frequencies[i]
        total_time = len(data[i][1])/sample_freq
        N_div = len(data[i][1])//N_samples
        for num in range(N_div):
            if (((total_time/N_div)*(num) <= onset) and ((total_time/N_div)*(num+1) > onset)) or (((total_time/N_div)*(num) >= onset) and ((total_time/N_div)*(num) < offset)):
                labels.append([1])
            else:
                labels.append([0])
    else:
        N_div = len(data[i][1])//N_samples
        for num in range(N_div):
            labels.append([0])
labels = np.reshape(labels, (1,-1))[0]

In [21]:
len(mont1_signal[8400])

2000

In [22]:
len(labels)

8498

In [32]:
np.unique(mont1_signal, return_counts=True)

(array([-51.61377435, -49.50124318, -47.74037637, ...,  43.29407623,
         43.53920519,  43.78575418]),
 array([1, 1, 1, ..., 1, 1, 1]))

In [29]:
oversample = SMOTE()
# downsampling
mont1_signal_smote, labels_smote = oversample.fit_resample(mont1_signal,labels)

In [33]:
np.unique(mont1_signal_smote, return_counts=True)

(array([-51.61377435, -49.50124318, -47.74037637, ...,  43.29407623,
         43.53920519,  43.78575418]),
 array([1, 1, 1, ..., 1, 1, 1]))

In [35]:
undersample = RandomUnderSampler()

# Erstellen Sie eine Funktion, die das Resampling durchführt
def resample_signal(signal, labels):
    # Anwenden von RandomUnderSampler
    signal_resampled, labels_resampled = undersample.fit_resample(signal, labels)
    return signal_resampled, labels_resampled

# Anwenden der Funktion auf jedes Signal
mont1_signal_resampled, labels_resampled_1 = resample_signal(np.array(mont1_signal), labels)
mont2_signal_resampled, labels_resampled_2 = resample_signal(np.array(mont2_signal), labels)
mont3_signal_resampled, labels_resampled_3 = resample_signal(np.array(mont3_signal), labels)

# Stellen Sie sicher, dass die Labels für alle Signale gleich sind, da sie das gleiche Set von Beispielen repräsentieren sollten
assert np.array_equal(labels_resampled_1, labels_resampled_2)
assert np.array_equal(labels_resampled_1, labels_resampled_3)

In [38]:
import numpy as np
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler
from wettbewerb import load_references, get_3montages, get_6montages
import mne
from scipy import signal as sig
from imblearn.under_sampling import RandomUnderSampler


# Daten laden und vorbereiten
training_folder  = "../../shared_data/training_mini"
ids, channels, data, sampling_frequencies, reference_systems, eeg_labels = load_references(training_folder)

number_montages = 3
N_samples = 2000 # Number of samples per division
# Decompose the wave
wavelet = 'db4'
scaler = StandardScaler()
new_signal = []

mont1_signal = []
mont2_signal = []
mont3_signal = []
whole_mont = [mont1_signal,mont2_signal,mont3_signal]
for i,_id in enumerate(ids):
    
    if number_montages == 6:
        _montage, _montage_data, _is_missing = get_6montages(channels[i], data[i])
    else:
        _montage, _montage_data, _is_missing = get_3montages(channels[i], data[i])
        
    _fs = sampling_frequencies[i]
    features_per_id = []

    for j, signal_name in enumerate(_montage):
        signal = _montage_data[j]
        # Notch-Filter to compensate net frequency of 50 Hz
        signal_notch = mne.filter.notch_filter(x=signal, Fs=_fs, freqs=np.array([50.,100.]), n_jobs=2, verbose=False)
        # Bandpassfilter between 0.5Hz and 70Hz to filter out noise
        signal_filter = mne.filter.filter_data(data=signal_notch, sfreq=_fs, l_freq=0.5, h_freq=70.0, n_jobs=2, verbose=False)
        # Defining number of divisions for signal
        N_div = len(signal_filter)//N_samples
        # Normalizing data
        norm_montage_data = scaler.fit_transform(signal_filter.reshape(-1,1)).reshape(1,-1)[0]
    
        for i in range(N_div):
            montage_array = norm_montage_data[i*N_samples:(i+1)*N_samples]
            whole_mont[j].append(montage_array)



labels = []
for i,_id in enumerate(ids):
    if eeg_labels[i][0]:
        onset = eeg_labels[i][1]
        offset = eeg_labels[i][2]
        sample_freq = sampling_frequencies[i]
        total_time = len(data[i][1])/sample_freq
        N_div = len(data[i][1])//N_samples
        for num in range(N_div):
            if (((total_time/N_div)*(num) <= onset) and ((total_time/N_div)*(num+1) > onset)) or (((total_time/N_div)*(num) >= onset) and ((total_time/N_div)*(num) < offset)):
                labels.append([1])
            else:
                labels.append([0])
    else:
        N_div = len(data[i][1])//N_samples
        for num in range(N_div):
            labels.append([0])
labels = np.reshape(labels, (1,-1))[0]


# Instanziierung von RandomUnderSampler
undersample = RandomUnderSampler()

# Erstellen Sie eine Funktion, die das Resampling durchführt
def resample_signal(signal, labels):
    # Anwenden von RandomUnderSampler
    signal_resampled, labels_resampled = undersample.fit_resample(signal, labels)
    return signal_resampled, labels_resampled

# Anwenden der Funktion auf jedes Signal
mont1_signal_resampled, labels_resampled_1 = resample_signal(np.array(mont1_signal), labels)
mont2_signal_resampled, labels_resampled_2 = resample_signal(np.array(mont2_signal), labels)
mont3_signal_resampled, labels_resampled_3 = resample_signal(np.array(mont3_signal), labels)

# Stellen Sie sicher, dass die Labels für alle Signale gleich sind, da sie das gleiche Set von Beispielen repräsentieren sollten
assert np.array_equal(labels_resampled_1, labels_resampled_2)
assert np.array_equal(labels_resampled_1, labels_resampled_3)

labels_resampled = labels_resampled_1

whole_mont_resampled = [mont1_signal_resampled,mont2_signal_resampled,mont3_signal_resampled]



whole_mont_resampled_np = np.array(whole_mont_resampled)

# Dataset-Klasse
class EEGDataset(Dataset):
    def __init__(self, whole_mont_resampled_np, labels_resampled):
        # Hier wird angenommen, dass `data` bereits ein Tensor ist. Wenn nicht, sollten Sie `data` in einen Tensor umwandeln.
        self.data = torch.from_numpy(whole_mont_resampled_np).float()
        self.labels = torch.from_numpy(np.array(labels_resampled)).long()

    def __len__(self):
        return self.data.shape[1]  # Anzahl der Beispiele entspricht nun dem zweiten Dimension

    def __getitem__(self, idx):
        # Für jedes Beispiel: Holt das idx-te Beispiel über alle Kanäle
        # Keine Notwendigkeit, permute oder unsqueeze zu verwenden
        sample = self.data[:, idx, :]  # Behält die Form [Kanäle, Länge] bei
        label = self.labels[idx]
        return sample, label

# Initialisierung des EEGDataset mit vorbereiteten Daten und Labels
eeg_dataset = EEGDataset(whole_mont_resampled_np, labels_resampled)



# CNN-Modell
class CNN(nn.Module):
    def __init__(self, num_classes, seq_length):
        super().__init__()
        self.classifier = nn.Sequential(
            nn.Conv1d(in_channels=3, out_channels=6, kernel_size=5),
            nn.BatchNorm1d(num_features=6),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2),
            nn.Conv1d(6, 16, 5),
            nn.BatchNorm1d(16),
            nn.ReLU(),
            nn.MaxPool1d(2, 2),
        )

        # Anpassung für die Berechnung der Größe des linearen Layers
        linear_input_size = self._get_conv_output(seq_length)

        self.fc = nn.Sequential(
            nn.Linear(linear_input_size, 120),
            nn.BatchNorm1d(120),
            nn.ReLU(),
            nn.Linear(120, 84),
            nn.BatchNorm1d(84),
            nn.ReLU(),
            nn.Linear(84, num_classes),
        )

    def _get_conv_output(self, shape):
        with torch.no_grad():
            input = torch.zeros(1, 3, shape)
            output = self.classifier(input)
            return output.numel()

    def forward(self, x):
        x = self.classifier(x)
        x = torch.flatten(x, start_dim=1)
        x = self.fc(x)
        return x


# Teilen Sie Ihre Daten in Trainings- und Validierungssets
dataset_size = len(eeg_dataset)
train_size = int(dataset_size * 0.8)
val_size = dataset_size - train_size
train_dataset, val_dataset = random_split(eeg_dataset, [train_size, val_size])

train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=32)



import numpy as np
import torch

class EarlyStopping:
    def __init__(self, patience=5, verbose=True, delta=0.0, path='checkpoint.pt', trace_func=print):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.f1_score_max = np.Inf
        self.delta = delta
        self.path = path
        self.trace_func = trace_func

    def __call__(self, f1_score, model):
        score = f1_score

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(f1_score, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            self.trace_func(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(f1_score, model)
            self.counter = 0

    def save_checkpoint(self, f1_score, model):
        if self.verbose:
            self.trace_func(f'Validation F1 Score increased ({self.f1_score_max:.6f} --> {f1_score:.6f}). Saving model...')
        torch.save(model.state_dict(), self.path)
        self.f1_score_max = f1_score


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNN(num_classes=2, seq_length=2000).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.05, momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=100)

early_stopping = EarlyStopping(patience=5, verbose=True, delta=0.01, path='checkpoint.pt')

def train(model, train_dataloader, val_dataloader, criterion, optimizer, scheduler, epochs, device):
    for epoch in range(epochs):
        model.train()
        train_losses = []
        train_predictions = []
        train_targets = []

        for inputs, labels in train_dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_losses.append(loss.item())

            _, predictions = torch.max(outputs, 1)
            train_predictions.extend(predictions.cpu().numpy())
            train_targets.extend(labels.cpu().numpy())

        train_f1 = f1_score(train_targets, train_predictions, average='macro')

        val_losses = []
        val_predictions = []
        val_targets = []
        model.eval()
        with torch.no_grad():
            for inputs, labels in val_dataloader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_losses.append(loss.item())

                _, predictions = torch.max(outputs, 1)
                val_predictions.extend(predictions.cpu().numpy())
                val_targets.extend(labels.cpu().numpy())

        val_f1 = f1_score(val_targets, val_predictions, average='macro')

        print(f'Epoch {epoch+1}: Train F1: {train_f1:.4f}, Val F1: {val_f1:.4f}')

        early_stopping(val_f1, model)
        if early_stopping.early_stop:
            print("Early stopping")
            break

        scheduler.step()

train(model, train_dataloader, val_dataloader, criterion, optimizer, scheduler, epochs=100, device=device)

# Nach dem Training das beste Modell laden
model.load_state_dict(torch.load('checkpoint.pt'))



826	 Dateien wurden geladen.


/tmp/ipykernel_18692/871857422.py:41: RuntimeWarning: filter_length (2641) is longer than the signal (400), distortion is likely. Reduce filter length or filter a longer signal.
  signal_notch = mne.filter.notch_filter(x=signal, Fs=_fs, freqs=np.array([50.,100.]), n_jobs=2, verbose=False)
/tmp/ipykernel_18692/871857422.py:43: RuntimeWarning: filter_length (2641) is longer than the signal (400), distortion is likely. Reduce filter length or filter a longer signal.
  signal_filter = mne.filter.filter_data(data=signal_notch, sfreq=_fs, l_freq=0.5, h_freq=70.0, n_jobs=2, verbose=False)
/tmp/ipykernel_18692/871857422.py:41: RuntimeWarning: filter_length (2641) is longer than the signal (400), distortion is likely. Reduce filter length or filter a longer signal.
  signal_notch = mne.filter.notch_filter(x=signal, Fs=_fs, freqs=np.array([50.,100.]), n_jobs=2, verbose=False)
/tmp/ipykernel_18692/871857422.py:43: RuntimeWarning: filter_length (2641) is longer than the signal (400), distortion is

Epoch 1: Train F1: 0.8926, Val F1: 0.9406
Validation F1 Score increased (inf --> 0.940630). Saving model...
Epoch 2: Train F1: 0.9371, Val F1: 0.9238
EarlyStopping counter: 1 out of 5
Epoch 3: Train F1: 0.9477, Val F1: 0.9235
EarlyStopping counter: 2 out of 5
Epoch 4: Train F1: 0.9523, Val F1: 0.9649
Validation F1 Score increased (0.940630 --> 0.964893). Saving model...
Epoch 5: Train F1: 0.9579, Val F1: 0.9437
EarlyStopping counter: 1 out of 5
Epoch 6: Train F1: 0.9667, Val F1: 0.8575
EarlyStopping counter: 2 out of 5
Epoch 7: Train F1: 0.9717, Val F1: 0.9547
EarlyStopping counter: 3 out of 5
Epoch 8: Train F1: 0.9718, Val F1: 0.9342
EarlyStopping counter: 4 out of 5
Epoch 9: Train F1: 0.9760, Val F1: 0.9236
EarlyStopping counter: 5 out of 5
Early stopping


<All keys matched successfully>

In [2]:
print("Daten Typ:", type(whole_mont))  # Überprüft den Typ der Variable
if isinstance(whole_mont, np.ndarray):
    print("Daten Form:", whole_mont.shape)
else:
    # Für Listen oder andere Datentypen, die keine .shape-Attribut haben
    print("Länge der Daten:", len(whole_mont))
    if len(whole_mont) > 0:
        print("Typ des ersten Elements:", type(whole_mont[0]))
        print("Form des ersten Elements (falls NumPy Array):", np.array(whole_mont[0]).shape)

print("Labels Typ:", type(labels))
print("Labels Form:", np.array(labels).shape)


Daten Typ: <class 'list'>
Länge der Daten: 3
Typ des ersten Elements: <class 'list'>
Form des ersten Elements (falls NumPy Array): (8498, 2000)
Labels Typ: <class 'numpy.ndarray'>
Labels Form: (8498,)


In [17]:
print(type(whole_mont_np))
print(whole_mont_np.shape)


<class 'numpy.ndarray'>
(3, 8498, 2000)


In [12]:
np.unique(labels_, return_counts=True)

NameError: name 'labels_smote' is not defined

In [84]:
print(flattened_data_np.shape)

(25494, 2000)


In [85]:
whole_mont_np = np.array(whole_mont)

In [10]:
print(whole_mont_np.shape)

(3, 8498, 2000)


In [11]:
print(labels.shape)

(8498,)
